In [2]:
import bs4
import json
from enum import Enum, EnumMeta
import numpy as np
import pprint

# BUILD PROCESSOR DEV NOTEBOOK

## INTRODUCTION

L'algorithme de build processor construit le build process.  
Le build process contient la liste des actions basiques (movement, changement station ...) envoyées au robot pour effectuer l'opération demandée.  
Ces actions sont définies par les roboticiens et seront (à terme) stoquées dans une base de données.

La construction automatique du build process repose sur un système de dépendancies et de next 
Pour chaque action stoquées dans la base, le roboticiens définies des actions qui devront être réalisées avant l'action ciblée (les dépendancies) et celles qui devront être réalisées après (les next).

Exemple :  
Pour le perçages du fasteners `en6115v2-3.305`, l'action robot associée est une action de type WORK.DRILLING avec l'id 1234. 

Avant d'effectuer le perçage, le robot dois effectuer un déplacement sur le point a percer : cette action est une dependance de l'action de perçage.  

Avant de me déplacement sur le point, le robot doit effectuer une phase d'approche sur le rail ciblé : cette action est une dependance de l'action de déplacement ...  

Après l'actioin de perçage, un contrôle est éventuellement nécessaire : cette action est un next de l'action de perçage.  
...

![dependencies](./media/dependencies_next.png)

Donc pour une action donnée, le build processor identifie l'action robot associée, liste les actions dependantes et resultantes (dependencies et next) et construit un arbre d'action représentant le build process.

## DEFINITION D'UNE ACTION

L'action est une operation basique effectué par le robot
Elle est definie par les parametres suivants :
* un id unique
* le type de l'action (mouvement d'approche, changement de station ...)
* sa definition (un objet qui décris l'action en fonction de son type)
* ses dépendences (la liste des actions à réaliser avant)
* ses next (la liste des actions à réaliser après)
* une description (lisible par un humain)



In [109]:
class Action:
    def __init__(self, id, atype, definition, description, dependencies=[], anext=[]):
        self._id = id
        self._type = atype
        self._definition = definition
        self._dependencies = dependencies
        self._next = anext
        self._description = description
        
    def __repr__(self):
        return self._description

    @property
    def dependencies(self):
        return self._dependencies
        
    @dependencies.setter
    def dependencies(self,dl):
        self._dependencies = dl
    
    @property
    def next(self):
        return self._next
    
    @next.setter
    def next(self, nl):
        self._next = nl

    @property
    def id(self):
        return self._id
    
    @property
    def type(self):
        return self._type.name

    @property
    def definition(self):
        return self._definition
    
    @property
    def priority(self):
        return self._type.priority
    
    @property
    def description(self):
        return self._description

    def addDependence(self, action):
        self._dependencies.append(action)
    
    def addNext(self, action):
        self._next.append(action)


### LE TYPE

Le type de l'action a 2 fonctions :
* déclarer le type de l'action
* définir la priorité de l'action (son niveau dans l'arbre de process)

J'ai défini les types sous la forme d'enumerations à plusieurs niveaux respectant le format : `TYPE.ELEMENT.MODE`.  
J'associe à chaque type :
* un parametre de priorité, qui donne à l'action sa position dans l'arbre de process
* un nom

Ex : Mouvement du bras en approche :
* Enumeration => MOVE.ARM.APPROACH
* Priorité => 50
* Nom : "MOVE.ARM.APPROACH"

Pour gérer le routage vers les sous niveau et les paramêtres de priorité je définis 2 interfaces héritant de EnumMeta qui implémente les fonctions dont j'ai besoin.

In [110]:
class EnumLevelInterface(EnumMeta):
    """
    Interface pour prendre un charge l'enumeration à plusieurs niveau
    """
    def __getitem__(self, name):
        # surcharge de la function __getitem__ pour gerer l enumeration a plusieurs niveaux
        dot = name.find('.')
        if dot!=-1:
            cindex = name[:dot]
            oindex = name[(dot+1):]
            e = super().__getitem__(cindex).value
            return e.__getitem__(oindex)
        else:
            e = super().__getitem__(name)
            if EnumPriorityInterface in e.__class__.__bases__:
                return e
            else:
                return e.value
    
class EnumPriorityInterface(Enum, metaclass=EnumLevelInterface):
    """
    Interface de definition de l'objet Type
    """    
    def __init__(self, priority, name):
        # creation de l instance
        self._priority = priority
        self._name = name 

    @property
    def priority(self):
        # getter priority
        return self._priority
    @property
    def name(self):
        # getter  name
        return self._name


#### DEFINITION DES ENUMERATION TYPES

TYPE MOVE ARM

In [111]:
class ActionArmMove(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration MOVE.ARM avec 3 modes : APPROACH CLEARANCE WORK
    APPROACH = (50, 'MOVE.ARM.APPROACH')
    CLEARANCE = (50, 'MOVE.ARM.CLEARANCE')
    WORK = (60, 'MOVE.ARM.WORK')

TYPE MOVE STATION

In [112]:
class ActionStationMove(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration MOVE.STATION avec 2 modes : WORK HOME
    WORK = (40, 'MOVE.STATION.WORK')
    HOME = (40, 'MOVE.STATION.HOME')

TYPE WORK

In [113]:
class ActionWork(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration WORK avec 2 modes : DRILL FASTEN
    DRILL=(70, 'WORK.DRILL')
    FASTEN=(80, 'WORK.FASTEN')

TYPE LOAD/UNLOAD

In [114]:
class ActionLoad(EnumPriorityInterface, metaclass=EnumLevelInterface):
    # enumeration LOAD avec 2 elements EFFECTOR TOOL 
    EFFECTOR=(20, 'LOAD.EFFECTOR')
    TOOL=(30, 'LOAD.TOOL')

class ActionUnload(EnumPriorityInterface, metaclass=EnumLevelInterface):
    EFFECTOR=(20, 'UNLOAD.EFFECTOR')
    TOOL=(30, 'UNLOAD.TOOL')


#### DEFINITION DES ENUMERATION INTERMEDIAIRE

In [115]:
class ActionMove(Enum, metaclass=EnumLevelInterface):
    # sous enumeration MOVE avec 2 element ARM et station
    ARM = ActionArmMove
    STATION = ActionStationMove
    
class ActionType(Enum, metaclass=EnumLevelInterface):
    # enumeration racine avec 3 action LOAD UNLOAD MOVE WORK
    LOAD = ActionLoad 
    UNLOAD = ActionUnload
    MOVE = ActionMove
    WORK = ActionWork

### LA DEFINITION

On intègre dans l'attribut definition un objet decrivant l'action en fonction de son type.  
Ex : pour un objet de type MOVE.ARM.APPROACH on intègre un objet Movement décrivant le mouvement d'approche.

#### OBJET MOVEMENT

L'objet Movement décris un mouvement effectué par le bras robot ou la station.  
Il est défini par les paramêtres suivants :
* uf : l'id de la reference userframe robot
* ut : l'id de la reference usertool robot
* points : la liste des points de passage qui décrivent le mouvement.


In [116]:
class Movement:
    def __init__(self, uf, ut, points):
        self._uf = uf
        self._ut = ut
        self._points = points

#### OBJET POINT

L'objet Point représente un point de passage du mouvement.  
Il est défini par les paramêtres suivants :
* cnt : la precision de la trajectoire par rapport au point
* speed : la vitesse du bras (en %)
* position : objet position représentant les coordonnees du point (ou la position du bras) et les parametres du bras.
* path : object path représentant le type de trajectoire pour atteindre le point


In [117]:
class Point:
    def __init__(self, cnt, speed, path, position):
        self._cnt = cnt
        self._speed = speed
        self._position = position
        self._path = path

#### OBJET PATH

L'objet path represente le type de trajectoire pour atteindre le point.  
3 type sont possible :
* CIRCULAR
* LINEAR
* JOINT

Je definit mon objet Path sous la forme d'une enumeration avec en cle les 3 types sous le format "human readable" et l'instruction du programme fanuc en valeur.


In [118]:
class Path(Enum):
    CIRCULAR='C'
    LINEAR='L'
    JOINT='J'

#### LES OBJETS POSITION

L'objet Position représente la position du bras.  
2 types de représentations sont possibles :
* type JOINT : position du bras par les angles de chaque moteur (j1,j2,j3,j4,j5,j6)
* type CARTESIAN : position du bras par les coordonnées et orientation du Tool Frame. (x,y,z,w,p,r)

L'objet Position est défini par les paramêtres suivants :
* vector : le vecteur représentant la position
* type : type de position => JOINT ou CARTESIAN
* config : pour le type CARTESIAN, un objet représentant la configuration du bras (null pour JOINT)
* e1 : la position de la station sur le 7° axe

Je decline cette classe Position en 2 sous classes :
* PositionCrt : Position du bras de type CARTESIAN
* PositionJoint : Position du bras de type JOINT

Je défini le type de position sous la forme d'une enumeration avec en cle les 2 types sous le format "human readable" et l'instruction du programme fanuc en valeur.

In [119]:

class PositionType(Enum):
    JOINT='jnt'
    CARTESIAN='crt'

class Position:
    def __init__(self, pvector, ptype, e1, config):
        self._vector = pvector
        self._type = ptype
        self._config = config
        self._e1 = e1

class PositionCrt(Position):
    def __init__(self, pvector, e1, config):
        super(PositionCrt, self).__init__(pvector, PositionType.CARTESIAN, e1, config)
    
    def to_dict(self):
        keys = ['x', 'y','z', 'w', 'p', 'r']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        crt_dict = self.__dict__.copy()
        crt_dict['_vector'] = dict(tl)
        return crt_dict
    
    def get(self):
        keys = ['x', 'y','z', 'w', 'p', 'r']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        return {'vector':dict(tl), 'type':self._type.value, 'config':Serializer.to_dict(self._config), 'e1':float(self._e1)}
            
class PositionJoint(Position):
    def __init__(self, pvector, e1):
        super(PositionJoint, self).__init__(pvector, PositionType.JOINT, e1, None)
    
    def to_dict(self):
        keys = ['j1', 'j2','j3', 'j4', 'j5', 'j6']
        tl = [(keys[i], float(val)) for i, val in enumerate(self._vector.tolist())]
        jnt_dict = self.__dict__.copy()
        jnt_dict['_vector'] = dict(tl)
        return jnt_dict
      

#### OBJET CONFIGURATION

L'objet configuration représente la configuration defini (imposée) du bras pour le calcul des déplacement en mode CARTESIAN.
Il est défini par les paramêtres suivants :
* wrist : config du poignet => FLIP ou NOFLIP
* forearm : config de l'avant bras => UP ou DOWN
* arm : config du bras => TOWARD ou BACKWARD
* j4, j5, j6 : debattement angulaire des axe j4, j5 et j6 toujours à 0 pour notre besoin

Je défini la config pour chaque element (wrist, forearm, arm) sous la forme d'une enumeration avec en cle les options sous le format "human readable" et l'instruction du programme fanuc en valeur.

In [120]:
class Configuration:
    def __init__(self, wrist, forearm, arm):
        self._wrist = wrist
        self._forearm = forearm
        self._arm = arm
        self._j4 = 0
        self._j5 = 0
        self._j6 = 0

class WristConfig(Enum):
    FLIP='F'
    NOFLIP='N'

class ForeArmConfig(Enum):
    UP='U'
    DOWN='D'

class ArmConfig(Enum):
    TOWARD='T'
    BACKWARD='D'

### CLASSE DE SERIALISATION

Ma classe Serializer contient toutes les fonctions pour la [sérialisation](https://docs.microsoft.com/fr-fr/dotnet/csharp/programming-guide/concepts/serialization/#:~:text=La%20s%C3%A9rialisation%20est%20le%20processus,processus%20inverse%20est%20appel%C3%A9%20d%C3%A9s%C3%A9rialisation.) de mes objets pour un enregistrement en base ou la génération du build process.

A terme, elle contiendra également les fonctions de déserialisation.


In [121]:
class Serializer:
    # fonction de serialisation des objets
    @staticmethod
    def serialize(obj, form="json", database=False):
        if(form =="json"):
            so = Serializer.to_dict(obj, database)
            return json.dumps(so)
    
    @staticmethod
    def to_dict(obj, database=False):
        if obj.__class__.__module__ == 'builtins':
            if type(obj) == list:
                l=[] 
                for va in obj:
                    l.append(Serializer.to_dict(va, database))
                return l
            elif type(obj) == dict:
                d={} 
                for key, val in obj.items():
                    k = key.replace('_','')
                    v = Serializer.to_dict(val, database)   
                    d[k]=v 
                return d
            else :
                return Serializer.__cast_val(obj)
            
        elif getattr(obj, 'to_dict', None):
            return  Serializer.to_dict(obj.to_dict(), database)
        
        elif EnumPriorityInterface in obj.__class__.__bases__ or Enum in obj.__class__.__bases__:
            if database : 
                return obj.name
            else:
                return obj.value
        else:
            so = {}
            for key, val in obj.__dict__.items():
                k = key.replace('_','')
                v = Serializer.to_dict(val, database)   
                so[k]=v
            return so
        
        
    @staticmethod
    def __cast_val(val):
        if isinstance(val, str) and val.isnumeric():
            return int(val) if val.isdecimal() else float(val)
        else:
            return val

## MISE EN APPLICATION 

### Creation d'une Action en manuel

Definition d'un déplacement de type work sur le point id en6115v2-3.305.
Et serialisation pour enregistrement en database

Rappel du xml :
```xml
<Phase id="4" Type="position" Info="Drill" FastName="en6115V2-3.305" Mode="crt">
                <Point id="1" UT="2" UF="1" X="751.5" Y="1987" Z="-57.75" W="-180" P="0" R="-70" E1="1538" t4="0" t5="0" t6="0" front="true" up="true" left="false" flip="true" type="linear" speed="100" approx="fine" cnt="100" />
            </Phase>
```


In [122]:
# Definition de la position du bras
vector = np.array((751.5, 1987,-57.75,-180, 0.0, -70)) # vecteur de position  
e1 = 1538.0 # position 7 axe
config = Configuration( 
    WristConfig.FLIP,
    ForeArmConfig.UP,
    ArmConfig.TOWARD
) #config FUT 000

# definition d'un objet position type Cartesian
position = PositionCrt(vector, e1, config) 

# definition du point de passage 

cnt = 100 # precision 100
speed = 100 # vitesse 100%
path = Path.LINEAR # forme trajectoire
point = Point(cnt, speed, path, position)

# definition du mouvement

ut = 1 # user tool id
uf = 1 # user frame id
movement = Movement(ut, uf, [point]) 

# definition de l'action
action_id = 'XXXX'
action_type =  ActionType['MOVE']['ARM']['WORK']
action_desc = 'move to fastener en6115v2-3.305'
action_dependencies = []
action_next = []

action = Action(action_id, action_type, movement, action_desc, action_dependencies, action_next)

In [123]:
# affichage de la version serialise pour enregistrement en database
pprint.pprint(Serializer.to_dict(action, database=True))

{'definition': {'points': [{'cnt': 100,
                            'path': 'LINEAR',
                            'position': {'config': {'arm': 'TOWARD',
                                                    'forearm': 'UP',
                                                    'j4': 0,
                                                    'j5': 0,
                                                    'j6': 0,
                                                    'wrist': 'FLIP'},
                                         'e1': 1538.0,
                                         'type': 'CARTESIAN',
                                         'vector': {'p': 0.0,
                                                    'r': -70.0,
                                                    'w': -180.0,
                                                    'x': 751.5,
                                                    'y': 1987.0,
                                                    'z': -57.75}},
                   

### Creation d'une liste d'action à partir du xml existant

Definition de la liste d'actions décrivant les phases d'approche, work et clearance pour la premiere station du fichier xml.

#### Definition des fonctions de parsing

In [124]:
def getPath(xml_path):
    path = None
    if xml_path =="linear":
        path = Path.LINEAR
    elif xml_path == "joint":
        path = Path.JOINT
    else:
        path = Path.CIRCULAR
    return path


#vecteur de position
def getMovementFromXMLPhases(bsxml_phase):
    
    #get phase attributes
    phat = bsxml_phase.attrs
    
    #get bs points 
    points = bsxml_phase.find_all('point')
    
    ps=[]
    uf = points[0].attrs.get('uf')
    ut = points[0].attrs.get('ut')
    
    for p in points:
        ps.append(getPointFromXMLPoint(p, phat['mode']))

    return Movement(uf, ut, ps)


def getPointFromXMLPoint(point, pos_type):
    
    poat = point.attrs
    path = getPath(poat['type'])
    
    cnt = poat.get('cnt')
    speed = poat.get('speed')
    
    vector = np.array([
            poat['x'],
            poat['y'],
            poat['z'],
            poat['w'],
            poat['p'],
            poat['r']])

    config = None
    
    if poat.get("front") and poat.get("up") and poat.get("left") and poat.get("flip"):
        config = Configuration(WristConfig.FLIP if bool(poat.get("flip")) else WristConfig.NOFLIP,
                               ForeArmConfig.UP if bool(poat.get("up")) else ForeArmConfig.DOWN,
                               ArmConfig.TOWARD if bool(poat.get("front")) else ArmConfig.BACKWARD
                               )

    e1 = poat.get('e1')

    if pos_type =="joint":
        position = PositionJoint(vector, e1)
    elif pos_type == 'crt':
        position = PositionCrt(vector, e1, config)
    else :
        position = None
    
    return Point(cnt, speed, path, position)
    

def getActionFromXMLStation(bsxml_station):
    
    actions = []
    phases = bsxml_station.find_all('phase')
    
    for p in phases:
        mov = getMovementFromXMLPhases(p)
        if p.attrs['info'] == 'Approach':
            atype = ActionType['MOVE']['ARM']['APPROACH']
        elif p.attrs['info'] == 'Clearance':
            atype = ActionType['MOVE']['ARM']['CLEARANCE']
        elif p.attrs['info'] == 'Drill':
            atype = ActionType['MOVE']['ARM']['WORK']
        else:
            assert "action type error"

        actions.append(Action(0, atype, mov, p.attrs['info'], [], [])) 
        #id, atype, definition, description, dependencies, anext
        
    return actions
    

#### Parsing du XML et creation de la liste d'actions

In [125]:
# lecture du fichier
with open('./data/BuildProcess.xml', 'r') as f:
    txt = f.readlines()
    txt = ''.join(txt)

xbp = bs4.BeautifulSoup(txt, 'lxml') # parsing du xml avec bs
steps = xbp.find_all('step') # recuperation de la liste des steps
stations = steps[0].find_all('station') # recuperation des station du step 1

la = getActionFromXMLStation(stations[0]) # definition de la liste des actions en utilisant la fonction de parsing

#### Serialisation 

In [126]:

# serialisation des actions (sur les 5 premiere)
Serializer.serialize(la, database=True)

'[{"id": 0, "type": "MOVE.ARM.APPROACH", "definition": {"uf": 1, "ut": 1, "points": [{"cnt": 100, "speed": 100, "position": {"vector": {"j1": 0.0, "j2": 25.0, "j3": -40.0, "j4": 0.0, "j5": 113.0, "j6": 0.0}, "type": "JOINT", "config": null, "e1": 700}, "path": "JOINT"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 20.0, "j2": 35.0, "j3": -30.0, "j4": -40.0, "j5": 105.0, "j6": 90.0}, "type": "JOINT", "config": null, "e1": 700}, "path": "JOINT"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 40.0, "j2": 45.0, "j3": -15.0, "j4": -60.0, "j5": 105.0, "j6": 90.0}, "type": "JOINT", "config": null, "e1": 700}, "path": "JOINT"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 60.0, "j2": 60.0, "j3": -10.0, "j4": -80.0, "j5": 60.0, "j6": 110.0}, "type": "JOINT", "config": null, "e1": 700}, "path": "JOINT"}, {"cnt": 100, "speed": 100, "position": {"vector": {"j1": 60.0, "j2": 53.0, "j3": -2.0, "j4": -92.0, "j5": 47.0, "j6": 137.0}, "type": "JOINT", "config": null, 

## CONSTRUCTION DU BUILD PROCESS

Pour la construction du build process j'utilise un algorithme de construction d'arbre (Tree).  

Dans l'environnement python, j'utilise ***[treelib](https://treelib.readthedocs.io/en/latest/index.html)***.

In [127]:
from treelib import Tree, Node, plugins # import des Objets de treelib

**Cet algorithmes de construction d'arbres reposent sur 2 objets principaux :
* objet Tree => l'arbre
* objet Node => les noeuds de l'arbre

Pour chaque noeud intégré dans l'arbre, on défini noeuds parent.
Cette relation parent/enfants entre les noeuds nous permet de creer des arborescences de noeuds.

Couplé au système de dépendances, cet algorithme va me permettre de définir, pour effectuer l'opération ciblé (ex : Drilling en6115v2-3.305), une arborescence d'actions représentant l'arbre de construction du build process.

Donc pour une operation ciblé (ex : Drilling en6115v2-3.305), voici les étapes de construction du build process :
* récupérer l'action qui represente l'operation (en database).
* lister ses dépendances et les dépendances de ses dépendances.
* trier les action par priorité (lié au type d'action => changement station > movement d'approche).
* ajouter un noeud pour chaque actions dans un arbre, les dependances de l'actions representent les parents du noeuds (plus ou moins grand en fonction de la priorité)
* construire le build process (serialisation) suivant l'arbre obtenu.


### DEFINITION DES OBJETS

Pour notre besoin je vais definir :
* un object custo ActionNode qui herite de treelib.Node représentant un Noeuds contenant une actions
* un objet custo ActionTree qui herite de l'objet treelib.Tree représentant un Arbre d'actions. 

In [157]:
import operator

In [341]:

class ActionNode(Node):
    # ActionNode, herite de Node
    def __init__(self, action=None):
        super(ActionNode, self).__init__(action.description if action else 'root', action.id if action else 0, data=action)
    
    def __lt__(self, node):
        return True if self.priority > node.priority else False

    def __gt__(self, node):
        return True if self.priority < node.priority else False
    
    def __eq__(self, node):
        return True if self.priority == node.priority else False
    
    @property
    def priority(self):
        if self.data:
            return self.data.priority
        else:
            return 0
            
class ActionTree(Tree):
    
    def __init__(self):
        super(ActionTree, self).__init__()
        self.add_node(ActionNode())


    def add_action_node(self, action_node, parent):
        if not self.contains(action_node.identifier):
            self.add_node(action_node, parent = parent)

    def add_actions(self, actions):
        current = self.get_node(0)
        action_nodes = [ActionNode(an) for an in actions]
        
        for an in action_nodes:
            parent = self.__getParent(an , current)
            self.add_action_node(an, parent)
            current = an

    def __getParent(self, actual_node, previous_node):
        
        if actual_node == previous_node:
            parent = self.parent(previous_node.identifier)

        elif actual_node < previous_node:
            parent = previous_node

        else:
            parent = self.__getParent(actual_node, self.parent(previous_node.identifier))
                
        return parent
        
    

class Dependencies:  
    @staticmethod
    def listDependances(action):
        alist = Dependencies.__listDependances(action, action.priority)
        alist.sort(key=lambda item : item[0])
        return [a[1] for a in alist]
        #return alist
    
    @staticmethod
    def __listDependances(action, dn=0):
        alist = [(dn, action)]
        for a in action.dependencies:
            alist.extend(Dependencies.__listDependances(a, a.priority))
        for a in action.next:
            alist.extend(Dependencies.__listDependances(a, 9999-a.priority))
        return alist
    

### MISE EN APPLICATION

#### Definition de l'action de drilling

In [342]:
# definition de l'action de drilling

action_id = '1234'
action_type =  ActionType['WORK']['DRILL']
action_desc = 'drill fastener asna2392-3-04.178'
drill = Action(action_id, action_type, {}, action_desc)

#### Definition de l'action de mouvement work 

L'action est defini par le xml suivant :

```xml 
<Phase id="2" Type="position" Info="Drill" FastName="asna2392-3-04.178" Mode="crt">
                <Point id="1" UT="1" UF="1" X="810.3" Y="1360" Z="-27.1" W="90" P="-90" R="0" E1="700" t4="0" t5="0" t6="0" front="true" up="true" left="false" flip="true" type="linear" speed="100" approx="fine" cnt="100" />
            </Phase>
```        

In [368]:
# definition de l'action de mouvement

txt_mvt = '<Phase id="2" Type="position" Info="Drill" FastName="asna2392-3-04.178" Mode="crt"><Point id="1" UT="1" UF="1" X="810.3" Y="1360" Z="-27.1" W="90" P="-90" R="0" E1="700" t4="0" t5="0" t6="0" front="true" up="true" left="false" flip="true" type="linear" speed="100" approx="fine" cnt="100" /></Phase>'

mvt_parse= bs4.BeautifulSoup(txt_mvt, 'lxml')
work_mvt = getMovementFromXMLPhases(mvt_parse.phase)
work_action = Action('5678', ActionType['MOVE']['ARM']['WORK'], work_mvt, 'move to fastener asna2392-3-04.178')


#### Definition de l'action de mouvement approach

In [369]:
txt_mvt = '<Phase id="1" Type="position" Info="Approach" Mode="joint"><Point id="1" UF="1" UT="1" X="0" Y="25" Z="-40" W="0" P="113" R="0" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="2" UF="1" UT="1" X="20" Y="35" Z="-30" W="-40" P="105" R="90" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="3" UF="1" UT="1" X="40" Y="45" Z="-15" W="-60" P="105" R="90" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="4" UF="1" UT="1" X="60" Y="60" Z="-10" W="-80" P="60" R="110" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="5" UF="1" UT="1" X="60" Y="53" Z="-2" W="-92" P="47" R="137" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="6" UF="1" UT="1" X="60.1" Y="53.2" Z="-2" W="-120" P="25" R="150" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="7" UF="1" UT="1" X="60.1" Y="53.2" Z="-5.6" W="-169" P="24.9" R="222" E1="700" type="joint" speed="100" approx="fine" cnt="100" /><Point id="8" UF="1" UT="1" X="57.9" Y="57.9" Z="-6.9" W="-173.1" P="23.3" R="226.9" E1="700" type="joint" speed="100" approx="fine" cnt="100" /></Phase>'

mvt_parse= bs4.BeautifulSoup(txt_mvt, 'lxml')
approach_mvt = getMovementFromXMLPhases(mvt_parse.phase)
approach_action = Action('9101', ActionType['MOVE']['ARM']['APPROACH'], approach_mvt, 'approach move')


Definition de l'action de mouvement clearance

In [370]:
txt_mvt = '<Phase id="13" Type="position" Info="Clearance" Mode="joint"><Point id="1" UF="1" UT="1" X="57.9" Y="57.9" Z="-6.9" W="-173.1" P="23.3" R="226.9" E1="700" type="joint" speed="100" approx="fine" cnt="100" /><Point id="2" UF="1" UT="1" X="60.1" Y="53.2" Z="-5.6" W="-169" P="24.9" R="222" E1="700" type="joint" speed="100" approx="fine" cnt="100" /><Point id="3" UF="1" UT="1" X="60.1" Y="53.2" Z="-2" W="-120" P="25" R="150" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="4" UF="1" UT="1" X="60" Y="53" Z="-2" W="-92" P="47" R="137" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="5" UF="1" UT="1" X="60" Y="60" Z="-10" W="-80" P="60" R="110" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="6" UF="1" UT="1" X="40" Y="45" Z="-15" W="-60" P="105" R="90" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="7" UF="1" UT="1" X="20" Y="35" Z="-30" W="-40" P="105" R="90" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /><Point id="8" UF="1" UT="1" X="0" Y="25" Z="-40" W="0" P="113" R="0" E1="700" type="joint" speed="100" approx="cnt" cnt="100" /></Phase>'

mvt_parse= bs4.BeautifulSoup(txt_mvt, 'lxml')
clearance_mvt = getMovementFromXMLPhases(mvt_parse.phase)
clearance_action = Action('1112', ActionType['MOVE']['ARM']['CLEARANCE'], clearance_mvt, 'clearance move')


#### Definition de l'action de mouvement station

In [371]:
station_work_action = Action('1314', ActionType['MOVE']['STATION']['WORK'], {}, 'move station to rail')
station_home_action = Action('2223', ActionType['MOVE']['STATION']['HOME'], {}, 'move station to home')

#### Definition des actions chargement effecteur et outil

In [372]:
loadeff_action = Action('1516', ActionType['LOAD']['EFFECTOR'], {}, 'load effector')
loadtool_action = Action('1718', ActionType['LOAD']['TOOL'], {}, 'load tool')


Definition des actions dechargement effecteur et outil

In [373]:
unloadeff_action = Action('1920', ActionType['LOAD']['EFFECTOR'], {}, 'unload effector')
unloadtool_action = Action('2021', ActionType['LOAD']['TOOL'], {}, 'unload tool')

#### Creation des relations **dependencies** et **next**

In [374]:
# ajout des dependences de l'action drill => load effector, load tool et work movement
drill.dependencies = [loadeff_action, loadtool_action, work_action]

# ajout des dependances de l'action work movement => approach movement
work_action.dependencies = [approach_action]

# ajout des dependances de l'action approach movement => changement de station
approach_action.dependencies = [station_work_action]


In [375]:
# ajout des next de l'action approach => clearance movement
approach_action.next = [clearance_action]

# ajout des next de l'action load effector => unload effector
loadeff_action.next = [unloadeff_action]

# ajout des next de l'action load tool => unload tool
loadtool_action.next = [unloadtool_action]

# ajout des next de l'action move station work => move station home
station_work_action.next =  [station_home_action]

#### List des dependances et next

In [376]:
dnl = Dependencies.listDependances(drill)

In [377]:
# afichage de l'ensemble des action 
dnl

[load effector,
 load tool,
 move station to rail,
 approach move,
 move to fastener asna2392-3-04.178,
 drill fastener asna2392-3-04.178,
 clearance move,
 unload tool,
 unload effector,
 move station to home]

#### Construction de l'arbre 

In [378]:
#creation de l'arbre
process_tree = ActionTree()
# ajout de la liste d'actions
process_tree.add_actions(dnl)


In [379]:
process_tree.show()

root
├── load effector
│   ├── load tool
│   │   └── move station to rail
│   │       ├── approach move
│   │       │   └── move to fastener asna2392-3-04.178
│   │       │       └── drill fastener asna2392-3-04.178
│   │       └── clearance move
│   └── unload tool
├── unload effector
└── move station to home

